In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ls /content/gdrive/MyDrive/

 3.mp4		    demo		   non_human
 clip_test_video    DemoSprint_II	   nudity_test
 cluster_faces	    drugs		   sub.mp4
 Code.zip	    fk			   TestData
'Colab Notebooks'   frames		   transfer_learn_yamnet_model
 darknet	   'Getting started.pdf'   YamnetVideoClassification
'Deep Learning'     mf


In [6]:
%%capture
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [19]:
import torch
import clip
from PIL import Image
from itertools import islice
import glob


In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 131MiB/s]


In [11]:
def get_prediction(frame_path, list_of_labels, how_many_predictions) -> list:
    Highest3Predictions = []
    try:
        text = clip.tokenize(list_of_labels).to(device)
        image = preprocess(Image.open(frame_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()
            probs = probs.tolist()[0]
        vv = {}
        for i, j in enumerate(probs):
            vv[list_of_labels[i]] = j
        maxx = {k: v for k, v in sorted(vv.items(), key=lambda item: item[1], reverse=True)}
        Highest3Predictions = list(islice(maxx.items(), how_many_predictions))
        print(f"{frame_path} : {Highest3Predictions}")
    except:
        raise

    return Highest3Predictions

**If you want to test on images**

In [ ]:
framePath = "/content/gdrive/MyDrive/img_000050.png"

In [16]:
how_many_predictions = 3

In [17]:
list_of_labels = ["drugs", "smoking", "alcohol"]

In [29]:
res = get_prediction(framePath, list_of_labels, how_many_predictions)

/content/gdrive/MyDrive/img_000050.png : [('drugs', 0.7470703125), ('smoking', 0.1470947265625), ('alcohol', 0.10595703125)]


**If you want to test on folder of images**

In [25]:
folder_path =  "/content/gdrive/MyDrive/test/"

In [23]:
list_of_labels = ["drugs", "smoking", "alcohol"]

In [31]:
for image_path in glob.glob(folder_path + "*"):
  res = get_prediction(image_path, list_of_labels, how_many_predictions)

/content/gdrive/MyDrive/test/img_000014.png : [('drugs', 0.76025390625), ('alcohol', 0.138427734375), ('smoking', 0.101318359375)]
/content/gdrive/MyDrive/test/img_000015.png : [('drugs', 0.693359375), ('alcohol', 0.208251953125), ('smoking', 0.09832763671875)]
/content/gdrive/MyDrive/test/img_000020.png : [('drugs', 0.546875), ('smoking', 0.28369140625), ('alcohol', 0.16943359375)]
/content/gdrive/MyDrive/test/img_000019.png : [('drugs', 0.54248046875), ('alcohol', 0.328857421875), ('smoking', 0.1287841796875)]
/content/gdrive/MyDrive/test/img_000006.png : [('drugs', 0.54296875), ('smoking', 0.240966796875), ('alcohol', 0.2159423828125)]
/content/gdrive/MyDrive/test/img_000010.png : [('alcohol', 0.5107421875), ('drugs', 0.44384765625), ('smoking', 0.04534912109375)]
/content/gdrive/MyDrive/test/img_000001.png : [('drugs', 0.52734375), ('alcohol', 0.391845703125), ('smoking', 0.08087158203125)]
